# GUI - Nine Men Morris

In [1]:
%%HTML
<style>
.container { width:100% }
</style>

## Definitionen:
- ring = einer der 3 Quadrate (0-2)
- cell = ein punkt auf einem Ring (0-7)
- position = (ring, cell)
- board = Array aus 3 Arrays, die angeben, ob, und wenn ja welcher Stein dort sitzt
- remaining = \[noch nicht gesetzte Steine Spieler 1, noch nicht gesetzte Steine Spieler 2\]
- pieces = \[remaining, board\]

In [2]:
import ipycanvas
from ipycanvas import Canvas, MultiCanvas

## Mögliche Positionen

<img src="board_positions.png" alt="Mögliche Positionen" width="800"/>

In [3]:
positions = [([0.05, 0.05], [0.50, 0.05], [0.95, 0.05], [0.95, 0.50], [0.95, 0.95], [0.50, 0.95], [0.05, 0.95], [0.05, 0.50]),
             ([0.20, 0.20], [0.50, 0.20], [0.80, 0.20], [0.80, 0.50], [0.80, 0.80], [0.50, 0.80], [0.20, 0.80], [0.20, 0.50]),
             ([0.35, 0.35], [0.50, 0.35], [0.65, 0.35], [0.65, 0.50], [0.65, 0.65], [0.50, 0.65], [0.35, 0.65], [0.35, 0.50])]

dic_positions = {
    "a1":positions[0][6],
    "a4":positions[0][7],
    "a7":positions[0][0],
    "b2":positions[1][6],
    "b4":positions[1][7],
    "b6":positions[1][0],
    "c3":positions[2][6],
    "c4":positions[2][7],
    "c5":positions[2][0],
    "d1":positions[0][5],
    "d2":positions[1][5],
    "d3":positions[2][5],
    "d5":positions[2][1],
    "d6":positions[1][1],
    "d7":positions[0][1],
    "e3":positions[2][4],
    "e4":positions[2][3],
    "e5":positions[2][2],
    "f2":positions[1][4],
    "f4":positions[1][3],
    "f6":positions[1][2],
    "g1":positions[0][4],
    "g4":positions[0][3],
    "g7":positions[0][2]
}

## Initialisierung Canvas

**Aufbau Leinwand:**

board\[Hintergrund, Linien, Steine]

* BOARD_SIZE = Größe des Spielfeldes in Pixeln
* DOT_RADIUS = Radius der schwarzen kleinen Punkte, die mögliche Positionen markieren (in Abhängigkeit von der Spielfeldgröße)
* PIECE_RADIUS = Radius der Spielsteine
* COLOUR = Farben der [dots, pieces_player_1, pieces_player_2]

In [4]:
BOARD_SIZE = 400
DOT_RADIUS = BOARD_SIZE*0.025
PIECE_RADIUS = BOARD_SIZE*0.04
COLOUR = ['black', 'white', 'sienna']

#board[Hintergrund, Linien, Steine]
board = MultiCanvas(3, width = BOARD_SIZE, height = BOARD_SIZE)

# Hintergrund
board[0].fill_style = '#ffffcc'
board[0].fill_rect(0, 0, BOARD_SIZE)

# Strichstärke
board[1].line_width = 5

# Quadrate
board[1].stroke_rect(BOARD_SIZE*0.05, BOARD_SIZE*0.05, BOARD_SIZE*0.90) # Außenring
board[1].stroke_rect(BOARD_SIZE*0.20, BOARD_SIZE*0.20, BOARD_SIZE*0.60) # Mittelring
board[1].stroke_rect(BOARD_SIZE*0.35, BOARD_SIZE*0.35, BOARD_SIZE*0.30) # Innenring

# Linien
board[1].begin_path()
board[1].move_to(BOARD_SIZE*0.50, BOARD_SIZE*0.05) #oben
board[1].line_to(BOARD_SIZE*0.50, BOARD_SIZE*0.35)
board[1].move_to(BOARD_SIZE*0.95, BOARD_SIZE*0.50) #rechts
board[1].line_to(BOARD_SIZE*0.65, BOARD_SIZE*0.50)
board[1].move_to(BOARD_SIZE*0.50, BOARD_SIZE*0.95) #unten
board[1].line_to(BOARD_SIZE*0.50, BOARD_SIZE*0.65)
board[1].move_to(BOARD_SIZE*0.05, BOARD_SIZE*0.50) #links
board[1].line_to(BOARD_SIZE*0.35, BOARD_SIZE*0.50)
board[1].stroke()

# Punkte (außen, mitte, innen)
for ring in positions:
    for x,y in ring:
        board[1].fill_arc(BOARD_SIZE*x, BOARD_SIZE*y, DOT_RADIUS, 0, 360)

## Status des Spielfelds
* 0 = kein Stein
* 1 = weißer Stein
* 2 = brauner Stein

In [5]:
class status():
    pieces = [[9,9],[          # Anzahl zu setzender Steine (Spieler_1 (Weiß), Spieler_2 (Braun))
            [0,0,0,0,0,0,0,0], # äußerer Ring
            [0,0,0,0,0,0,0,0], # mittlerer Ring
            [0,0,0,0,0,0,0,0]  # innerer Ring
      ]]
    current_player = 1

## Spielsteine anzeigen

In [11]:
def draw_piece(ring, cell, player):
    board[2].fill_style = COLOUR[player]
    board[2].fill_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS, 0, 360)
    if player == 1:
        board[2].stroke_style = 'silver' 
    else:
        board[2].stroke_style = 'chocolate'
    board[2].stroke_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS, 0, 360)
    board[2].stroke_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS*0.7, 0, 360)
    board[2].stroke_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS*0.3, 0, 360)

In [7]:
def update_board(status):
    with ipycanvas.hold_canvas(board):
        board[2].clear()
        for ring in range(3):
            for cell in range(8):
                player = status.pieces[1][ring][cell]
                if player in [1, 2]: draw_piece(ring, cell, player)
        board[2].fill_style = 'black'
        board[2].font = '18px serif'
        board[2].fill_text('Spieler '+ str(status.current_player), BOARD_SIZE*0.4, BOARD_SIZE*0.45)
        board[2].fill_text('ist dran.', BOARD_SIZE*0.4, BOARD_SIZE*0.55)
    return board

In [8]:
def handle_mouse_down(x, y):
    global status
    for ring in positions:
        for pos in ring:
            if (pos[0]*BOARD_SIZE-PIECE_RADIUS/2 < x < pos[0]*BOARD_SIZE+PIECE_RADIUS/2 and pos[1]*BOARD_SIZE-PIECE_RADIUS/2 < y < pos[1]*BOARD_SIZE+PIECE_RADIUS/2):
                save_position(status, ring.index(pos), positions.index(ring))
    print("Ungültiger Zug. Bitte klicke auf eine mögliche Position.")
board[2].on_mouse_down(handle_mouse_down)

In [9]:
def save_position(status, pos, ring):
    status.pieces[1][ring][pos] = status.current_player
    #solange Anzahl zu seztender Steine > 0 =
    if(status.pieces[0][status.current_player-1]>0):
        status.pieces[0][status.current_player-1] = status.pieces[0][status.current_player-1]-1
    status.current_player = 2 if (status.current_player == 1) else 1;
    update_board(status)

In [10]:
update_board(status)

MultiCanvas(height=400, width=400)